# Projeto Final PLN 
Gabriel Cesário Silva Martins - 180100912 

Training and classifying the dataset with an Logistic Regression Model. 

# Mouting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ln --symbolic /content/drive/MyDrive/UnB/PLN/projeto_final/ /dir
%cd /dir/.
!ls .

/content/drive/MyDrive/UnB/PLN/projeto_final
LSTM_Classifier		       Mental-Health-Twitter-pre-processed.csv
Mental-Health-Twitter.csv      projeto_final
Mental-Health-Twitter.csv.zip


# Imports

In [52]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Reading the Dataset

In [ ]:
df = pd.read_csv('Mental-Health-Twitter-pre-processed.csv')
df = df.drop('Unnamed: 0', axis=1)

In [74]:
df.columns

Index(['post_id', 'post_created', 'post_text', 'user_id', 'followers',
       'friends', 'favourites', 'statuses', 'retweets', 'label', 'is_retweet',
       'tweet_num_words'],
      dtype='object')

In [ ]:
df.head(3)

,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label,is_retweet,tweet_num_words
0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,2 years since i diagnose anxiety depression to...,1013187241,84,211,251,837,0,1,0,16
1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,sunday i need break i plan spend little time p...,1013187241,84,211,251,837,1,1,0,11
2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,awake tire i need sleep my brain ideas,1013187241,84,211,251,837,0,1,0,8


In [78]:
df.drop(columns=['post_id', 'post_created', 'user_id'], axis=1, inplace=True)
df.head(3)

,post_text,followers,friends,favourites,statuses,retweets,label,is_retweet,tweet_num_words
0,2 years since i diagnose anxiety depression to...,84,211,251,837,0,1,0,16
1,sunday i need break i plan spend little time p...,84,211,251,837,1,1,0,11
2,awake tire i need sleep my brain ideas,84,211,251,837,0,1,0,8


In [83]:
df_labels = df['label']
df_labels.head(3)

0    1
1    1
2    1
Name: label, dtype: int64


In [86]:
df['label'] = df_labels

In [88]:
df.drop(columns='label', inplace=True)
df.head(3)

,post_text,followers,friends,favourites,statuses,retweets,is_retweet,tweet_num_words
0,2 years since i diagnose anxiety depression to...,84,211,251,837,0,0,16
1,sunday i need break i plan spend little time p...,84,211,251,837,1,0,11
2,awake tire i need sleep my brain ideas,84,211,251,837,0,0,8


# Separating the dataset

Making the training, validation and test datasets.

In [ ]:
random_seed = 168 # Same seed used for all models
x_train, x_rest, y_train, y_rest = train_test_split(df['post_text'], df['label'], random_state = random_seed, test_size=0.2, shuffle=True)

In [ ]:
x_valid, x_test, y_valid, y_test = train_test_split(x_rest, y_rest, random_state = random_seed, test_size=0.5, shuffle=True)

In [ ]:
print(f"Train: \n{x_train.head(3)}", f"\n\nTest: \n{y_train.head(3)}")

Train: 
530                                  could see grow loss
2181    therapy pinterest mindful eat depression depress
2165                                              google
Name: post_text, dtype: object 

Test: 
530     1
2181    1
2165    1
Name: label, dtype: int64


In [ ]:
vectorizer = TfidfVectorizer()

x_train_matrix = vectorizer.fit_transform(x_train)
x_train_matrix

<15943x16158 sparse matrix of type '<class 'numpy.float64'>'
	with 99762 stored elements in Compressed Sparse Row format>

In [ ]:
x_valid_matrix = vectorizer.transform(x_valid)
x_test_matrix = vectorizer.transform(x_test)
print("Shape of validation / test sets")
print(x_valid_matrix.shape, x_test_matrix.shape)

Shape of validation / test sets
(1993, 16158) (1993, 16158)


# The Logistic Regression Model
Making the model and training. Here, a Logistic Regression model with a stratified 10-fold cross validation is used. Cross-validation is used to estimate to estimate the parameter C, i.e., the value used to regulate the training.

In [70]:
model = LogisticRegressionCV(Cs=10, cv=10, scoring='accuracy', max_iter = 3000).fit(x_train_matrix, y_train)

In [71]:
# Testing the accuracy on the validation set
model.score(x_valid_matrix, y_valid)

0.782237832413447

## Testing

In [72]:
y_pred = model.predict(x_test_matrix)

In [73]:
print("\t-- Test Classification Results--\n")
print(classification_report(y_test, y_pred, digits=3))

	-- Test Classification Results--

              precision    recall  f1-score   support

           0      0.805     0.786     0.795      1017
           1      0.782     0.801     0.791       976

    accuracy                          0.793      1993
   macro avg      0.793     0.793     0.793      1993
weighted avg      0.794     0.793     0.793      1993



# Fontes
Abaixo, têm-se algumas das fontes utilizadas para execução deste notebook. Partes delas foram adaptadas aqui.

- https://medium.com/analytics-vidhya/applying-text-classification-using-logistic-regression-a-comparison-between-bow-and-tf-idf-1f1ed1b83640
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html